# Vision 기능

- GPT-4o with image inpus

- GPT-4o와 GPT-4 Turbo 모두 비전 기능을 갖추고 있습니다. 즉, 모델이 이미지를 촬영하고 그에 대한 질문에 답할 수 있습니다

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

이미지는 두 가지 주요 방법, 즉 이미지에 대한 링크를 전달하거나 요청에 base64로 인코딩된 이미지를 직접 전달하는 방법으로 모델에서 사용할 수 있습니다. 

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

### 이미지의 내용 설명
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

In [3]:
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "이 이미지의 내용을 설명해줘. 한국어로 설명해줘."},
        {"type": "image_url", "image_url": {"url": url}},
      ],
    }
  ],
  max_tokens=300,
)

response.to_dict()

{'id': 'chatcmpl-AwJV6L4PcwTT9S10MyI0TZgUu3EAm',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '이미지에는 넓은 초원과 그 위를 지나가는 나무 데크가 있습니다. 초원은 푸른 풀과 식물들로 가득 차 있으며, 주변에는 여러 나무와 덤불들이 보입니다. 하늘은 푸르르고 구름이 몇 개 떠 있으며, 전체적으로 평화롭고 자연의 아름다움을 느낄 수 있는 풍경입니다. 데크는 자연 속을 탐험할 수 있는 길로 이어지는 것을 나타냅니다.',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1738461348,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_bd83329f63',
 'usage': {'completion_tokens': 107,
  'prompt_tokens': 1160,
  'total_tokens': 1267,
  'completion_tokens_details': {'audio_tokens': 0,
   'reasoning_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1024}}}

In [4]:
print(response.choices[0].message.content)

이미지에는 넓은 초원과 그 위를 지나가는 나무 데크가 있습니다. 초원은 푸른 풀과 식물들로 가득 차 있으며, 주변에는 여러 나무와 덤불들이 보입니다. 하늘은 푸르르고 구름이 몇 개 떠 있으며, 전체적으로 평화롭고 자연의 아름다움을 느낄 수 있는 풍경입니다. 데크는 자연 속을 탐험할 수 있는 길로 이어지는 것을 나타냅니다.


### Base 64로 인코딩된 이미지 업로드
로컬에 이미지 또는 이미지 세트가 있는 경우 이를 Base 64 인코딩 형식으로 모델에 전달할 수 있습니다. 다음은 이에 대한 실제 예입니다.

In [8]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "data/냉장고내용물.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "이 이미지의 내용을 한국어로 설명해줘."},
        { "type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
      ],
    }
  ],
)

print(response.choices[0].message.content)

이 이미지는 냉장고 내부를 보여주고 있으며, 다양한 신선한 채소와 과일이 가득 차 있습니다. 상단 선반에는 딸기, 사과, 오렌지, 그리고 여러 가지 채소가 보이고, 중간 선반에도 여러 색깔의 채소와 과일이 정리되어 있습니다. 하단 서랍에는 당근, 호박과 같은 신선한 재료들이 있으며, 음료수병도 몇 개 눈에 띕니다. 전반적으로 건강식 재료들이 잘 정리된 모습입니다.


### 다중 이미지 입력
Chat Completions API는 base64로 인코딩된 형식이나 이미지 URL로 여러 이미지 입력을 가져와 처리할 수 있습니다. 모델은 각 이미지를 처리하고 모든 이미지의 정보를 사용하여 질문에 답합니다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg" width=300px />

<img src="https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg" width=300px />

In [6]:
response = client.chat.completions.create(
  model=Model,
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "이 이미지에는 무엇이 들어있나요? 그들 사이에 어떤 차이가 있나요?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://img.freepik.com/premium-photo/a-wooden-walkway-leads-to-a-green-field-with-a-mountain-in-the-background_800563-10576.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

In [7]:
print(response.choices[0].message.content)

두 이미지는 자연 경관을 보여주고 있으며, 공통점과 차이를 가지고 있습니다.

### 공통점:
1. **경관**: 둘 다 자연 환경에서 촬영된 장면이며, 모두 나무로 된 길이 있습니다.
2. **보행로**: 두 이미지 모두 보행로가 중앙에 배치되어 있어, 사람들이 걸어갈 수 있는 경로를 제공합니다.

### 차이점:
1. **환경**: 첫 번째 이미지는 풀밭과 나무, 푸른 하늘이 있는 습기 있는 지역을 보여줍니다. 두 번째 이미지는 넓은 논과 산이 보이며, 저녁 또는 아침의 따뜻한 빛을 받고 있습니다.
2. **색감과 분위기**: 첫 번째 이미지는 밝고 화사한 느낌을 주며, 두 번째 이미지는 더욱 음침하고 편안한 분위기를 연출합니다. 

이 두 이미지는 자연의 아름다움을 각기 다른 방식으로 표현하고 있습니다.
